<a href="https://colab.research.google.com/github/Devesh1602/BE-HPC/blob/main/Devesh-HPC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile bu.cu
#include <iostream>
#include <vector>
#include <cstdlib>  // For rand()
#include <ctime>    // For clock()

// Sequential Bubble Sort
void bubbleSortSequential(int* arr, int size) {
    for (int i = 0; i < size - 1; i++) {
        for (int j = 0; j < size - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

// Parallel Bubble Sort using CUDA
__global__ void bubbleSortParallel(int* arr, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

/*Inside the kernel, this line calculates the unique thread ID (tid) based on the block index (blockIdx.x) and the thread index (threadIdx.x).
In CUDA, threads are organized into blocks, and each block contains multiple threads. The thread ID is crucial for determining which elements of the array each thread will work on.*/

    if (tid < size) {
        for (int i = 0; i < size - 1; i++) {
            for (int j = 0; j < size - i - 1; j++) {
                if (arr[j] > arr[j + 1]) {
                    int temp = arr[j];
                    arr[j] = arr[j + 1];
                    arr[j + 1] = temp;
                }
            }
        }
    }
}
void printArray(int* arr, int size) {
    for (int i = 0; i < size; i++) {
        std::cout << arr[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    const int size = 1000;
    int arr[size];
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000;  // Random numbers between 0 and 999
    }

    /*
    //IF TO ADD CUSTOM USER INPUTS
    int size;
    std::cout << "Enter the size of the array: ";
    std::cin >> size;

    if (size <= 0) {
        std::cerr << "Invalid array size!" << std::endl;
        return 1;
    }

    int arr[size];
    std::cout << "Enter " << size << " integers for the array:" << std::endl;
    for (int i = 0; i < size; i++) {
        std::cin >> arr[i];
    }*/

    // Measure sequential bubble sort time
    clock_t startSeqBubble = clock();
    bubbleSortSequential(arr, size);
    clock_t endSeqBubble = clock();
    double timeSeqBubble = double(endSeqBubble - startSeqBubble) / CLOCKS_PER_SEC;

    // Measure parallel bubble sort time
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    clock_t startParBubble = clock();
    bubbleSortParallel<<<(size + 255) / 256, 256>>>(d_arr, size);
    cudaDeviceSynchronize();
    clock_t endParBubble = clock();
    double timeParBubble = double(endParBubble - startParBubble) / CLOCKS_PER_SEC;

    // Print sorted array after parallel sorting
    std::cout << "Parallelly Sorted Array: ";
    printArray(arr, size);

    // Print execution times
    std::cout << "Sequential Bubble Sort Time: " << timeSeqBubble << " seconds" << std::endl;
    std::cout << "Parallel Bubble Sort Time: " << timeParBubble << " seconds" << std::endl;

    // Free device memory
    cudaFree(d_arr);

    return 0;
}


Overwriting bu.cu


In [8]:
!nvcc bu.cu -o bu
!./bu

Parallelly Sorted Array: 0 0 2 2 4 6 8 9 10 11 11 12 16 17 17 18 19 21 21 21 21 22 23 25 27 27 28 29 30 30 30 31 32 33 34 36 36 36 39 40 41 42 42 42 43 43 46 47 49 49 50 51 52 53 54 55 57 57 57 58 59 59 59 60 62 63 67 67 69 71 71 72 73 74 74 80 81 81 81 81 81 82 82 84 84 84 84 86 87 87 88 90 90 90 91 93 94 95 97 97 97 98 99 100 100 105 107 109 109 114 115 116 117 117 120 121 122 122 123 124 124 125 126 126 127 127 127 127 128 130 131 131 133 134 134 135 136 137 139 139 142 143 144 147 149 149 150 151 152 153 154 154 155 157 157 157 159 159 163 167 168 170 171 171 172 172 172 173 176 177 178 179 179 179 181 181 181 183 184 188 189 189 189 190 190 190 193 195 195 196 197 197 198 199 199 202 202 204 205 205 205 206 207 209 210 211 211 211 214 215 217 217 218 218 219 222 223 224 224 225 226 226 227 228 228 228 229 231 231 231 232 232 233 234 234 235 235 236 237 237 238 244 245 248 249 250 253 253 254 255 255 258 259 260 261 262 262 266 266 269 269 269 270 270 270 272 273 274 275 276 278 27

In [9]:
%%writefile mer.cu
#include <iostream>
#include <vector>
#include <cstdlib>  // For rand()
#include <ctime>    // For clock()

// Sequential Merge Sort
void mergeSortSequential(int* arr, int left, int right) {
    if (left < right) {
        int mid = left + (right - left) / 2;
        mergeSortSequential(arr, left, mid);
        mergeSortSequential(arr, mid + 1, right);

        // Merge the sorted halves
        int temp[right - left + 1];
        int i = left, j = mid + 1, k = 0;

        while (i <= mid && j <= right) {
            if (arr[i] <= arr[j]) {
                temp[k++] = arr[i++];
            } else {
                temp[k++] = arr[j++];
            }
        }

        while (i <= mid) {
            temp[k++] = arr[i++];
        }

        while (j <= right) {
            temp[k++] = arr[j++];
        }

        for (int p = 0; p < k; p++) {
            arr[left + p] = temp[p];
        }
    }
}

// Parallel Merge Sort using CUDA
__global__ void merge(int* arr, int left, int mid, int right, int* temp) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    int i = left + tid;
    int j = mid + 1 + tid;
    int k = tid;

    while (i <= mid && j <= right) {
        if (arr[i] <= arr[j]) {
            temp[k] = arr[i];
            i += blockDim.x * gridDim.x;
        } else {
            temp[k] = arr[j];
            j += blockDim.x * gridDim.x;
        }
        k += blockDim.x * gridDim.x;
    }

    while (i <= mid) {
        temp[k] = arr[i];
        i += blockDim.x * gridDim.x;
        k += blockDim.x * gridDim.x;
    }

    while (j <= right) {
        temp[k] = arr[j];
        j += blockDim.x * gridDim.x;
        k += blockDim.x * gridDim.x;
    }

    __syncthreads();

    // Copy merged elements back to the original array
    for (int idx = tid; idx < k; idx += blockDim.x * gridDim.x) {
        arr[left + idx] = temp[idx];
    }
}

void mergeSortParallel(int* arr, int left, int right, int* temp) {
    if (left < right) {
        int mid = left + (right - left) / 2;
        mergeSortParallel(arr, left, mid, temp);
        mergeSortParallel(arr, mid + 1, right, temp);
        merge<<<1, 1>>>(arr, left, mid, right, temp);
    }
}

// Host function to invoke parallel merge sort
void parallelMergeSort(int* arr, int size) {
    int* d_arr;
    int* d_temp;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMalloc(&d_temp, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    int left = 0;
    int right = size - 1;

    mergeSortParallel(d_arr, left, right, d_temp);
    cudaDeviceSynchronize();

    cudaMemcpy(arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    cudaFree(d_temp);
}
void printArray(int* arr, int size) {
    for (int i = 0; i < size; i++) {
        std::cout << arr[i] << " ";
    }
    std::cout << std::endl;
}


int main() {
    const int size = 1000;
    int arr[size];
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000;  // Random numbers between 0 and 999
    }

    /*
    //IF TO ADD CUSTOM USER INPUTS
    int size;
    std::cout << "Enter the size of the array: ";
    std::cin >> size;

    if (size <= 0) {
        std::cerr << "Invalid array size!" << std::endl;
        return 1;
    }

    int arr[size];
    std::cout << "Enter " << size << " integers for the array:" << std::endl;
    for (int i = 0; i < size; i++) {
        std::cin >> arr[i];
    }*/

    clock_t startSeqMerge = clock();
    mergeSortSequential(arr, 0, size - 1);
    clock_t endSeqMerge = clock();
    double timeSeqMerge = double(endSeqMerge - startSeqMerge) / CLOCKS_PER_SEC;

    clock_t startParMerge = clock();
    parallelMergeSort(arr, size);
    clock_t endParMerge = clock();
    double timeParMerge = double(endParMerge - startParMerge) / CLOCKS_PER_SEC;

    // Print sorted array after parallel sorting
    std::cout << "Parallelly Sorted Array: ";
    printArray(arr, size);

    // Print execution times
    std::cout << "Sequential Merge Sort Time: " << timeSeqMerge << " seconds" << std::endl;
    std::cout << "Parallel Merge Sort Time: " << timeParMerge << " seconds" << std::endl;

    return 0;
}


Writing mer.cu


In [10]:
!nvcc mer.cu -o mer
!./mer

Parallelly Sorted Array: 0 0 2 2 4 6 8 9 10 11 11 12 16 17 17 18 19 21 21 21 21 22 23 25 27 27 28 29 30 30 30 31 32 33 34 36 36 36 39 40 41 42 42 42 43 43 46 47 49 49 50 51 52 53 54 55 57 57 57 58 59 59 59 60 62 63 67 67 69 71 71 72 73 74 74 80 81 81 81 81 81 82 82 84 84 84 84 86 87 87 88 90 90 90 91 93 94 95 97 97 97 98 99 100 100 105 107 109 109 114 115 116 117 117 120 121 122 122 123 124 124 125 126 126 127 127 127 127 128 130 131 131 133 134 134 135 136 137 139 139 142 143 144 147 149 149 150 151 152 153 154 154 155 157 157 157 159 159 163 167 168 170 171 171 172 172 172 173 176 177 178 179 179 179 181 181 181 183 184 188 189 189 189 190 190 190 193 195 195 196 197 197 198 199 199 202 202 204 205 205 205 206 207 209 210 211 211 211 214 215 217 217 218 218 219 222 223 224 224 225 226 226 227 228 228 228 229 231 231 231 232 232 233 234 234 235 235 236 237 237 238 244 245 248 249 250 253 253 254 255 255 258 259 260 261 262 262 266 266 269 269 269 270 270 270 272 273 274 275 276 278 27